In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
data = open('shakespeare-Copy1.txt','r').read()

In [3]:
data[20:100]

"  1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose mi"

In [4]:
vocab = sorted(set(data))

In [5]:
len(vocab)

84

In [6]:
char_to_ind = {c:i for i,c in enumerate(vocab)}

In [7]:
len(char_to_ind)

84

In [8]:
# ind_to_char = { i:c for i,c in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [9]:
len(ind_to_char)

84

In [10]:
len(data)

5445609

In [11]:
encoded_text = [char_to_ind[c] for c in data]

In [12]:
encoded_text[:4]

[0, 1, 1, 1]

In [13]:
len(encoded_text)

5445609

In [14]:
# creating batches

In [15]:
seq_len = 120

In [16]:
char = tf.data.Dataset.from_tensor_slices(encoded_text)

In [17]:
for i in char.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [18]:
sequences = char.batch(seq_len+1,drop_remainder=True)

In [19]:
def create_seq_targets(seq):
    input_txt=seq[:-1]
    target_txt=seq[1:]
    return input_txt,target_txt

In [20]:
dataset = sequences.map(create_seq_targets)

In [21]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(120,), dtype=tf.int32, name=None), TensorSpec(shape=(120,), dtype=tf.int32, name=None))>

In [22]:
for input_txt,target_txt in dataset.take(1):
    print(len(input_txt.numpy()))
    print(''.join(ind_to_char[input_txt.numpy()]))
    print(''.join(ind_to_char[target_txt.numpy()]))

120

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [23]:
# Batch size
batch_size = 120

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(seq_len, drop_remainder=True)

In [24]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(120, 120), dtype=tf.int32, name=None), TensorSpec(shape=(120, 120), dtype=tf.int32, name=None))>

In [25]:
len(dataset)

375

In [62]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
emb_size = 64

# Number of RNN units
rnn_size = 1026

In [27]:
from tensorflow.keras.layers import Dense,LSTM,Embedding,Dropout,Dropout,GRU

In [28]:
from tensorflow.keras.models import Sequential

In [29]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
def sparse_call(yactual,ypred):
    return sparse_categorical_crossentropy(yactual,ypred,from_logits=True)

In [54]:
def create_model(rnn_size,emb_size,batch_size,vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))

    model.compile(optimizer='adam',loss=sparse_call)
    return model

In [41]:
help(Embedding)

Help on class Embedding in module keras.src.layers.core.embedding:

class Embedding(keras.src.engine.base_layer.Layer)
 |  Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, sparse=False, **kwargs)
 |  
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  
 |  e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`
 |  
 |  This layer can only be used on positive integer inputs of a fixed range. The
 |  `tf.keras.layers.TextVectorization`, `tf.keras.layers.StringLookup`,
 |  and `tf.keras.layers.IntegerLookup` preprocessing layers can help prepare
 |  inputs for an `Embedding` layer.
 |  
 |  This layer accepts `tf.Tensor`, `tf.RaggedTensor` and `tf.SparseTensor`
 |  input.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.Sequential()
 |  >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
 |  >>> # The model will take as

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (120, None, 64)           5376      
                                                                 
 gru_1 (GRU)                 (120, None, 1026)         3361176   
                                                                 
 dense_1 (Dense)             (120, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
sample_model = create_model(rnn_size,emb_size,120,vocab_size)
predictions=[]

for input_txt,target_txt in dataset.take(1):
    predictions =sample_model(input_txt)

In [81]:
predictions.shape

TensorShape([120, 120, 84])

In [88]:
sample_indices= tf.random.categorical(predictions[0],num_samples=1)

In [89]:
sample_indices = tf.squeeze(sample_indices,axis=-1).numpy()

In [90]:
ind_to_char[sample_indices]

array(['B', 'V', 'I', 'd', '7', 'p', 'l', 't', 'N', 'N', '9', '0', 'S',
       'h', 'K', 'b', 'p', '-', 'u', 'j', 'J', 'p', 'S', '\n', '5', 'D',
       '?', 'q', 'H', 'M', 'K', 'b', '6', 'q', ')', 'q', '}', '[', '}',
       'L', 'D', 'l', 'P', 'H', '7', 'H', 'L', '8', '0', '`', 'A', 'i',
       'x', 'q', 'x', 'p', 'p', '?', 'Y', 'o', 'm', '\n', ']', 'L', 'T',
       '[', 'd', '7', 'U', 'B', 'D', 'b', 'x', 'i', ';', '0', 'x', '5',
       'g', 'q', 'W', 'Z', '>', 'l', '3', 'c', ' ', 'M', '8', '?', '6',
       'V', 'h', 'C', ':', 'P', ']', 'U', '<', ':', '?', 'R', '[', 'S',
       '|', 'D', '7', 'G', 'g', '|', '&', 'j', '5', 'l', 'H', '4', 'U',
       'v', 'P', 'p'], dtype='<U1')

In [48]:
# model.fit(dataset,epochs=30)

Epoch 1/30
375/375 [==============================] - 984s 3s/step - loss: 2.5081
Epoch 2/30
375/375 [==============================] - 908s 2s/step - loss: 1.7017
Epoch 3/30
375/375 [==============================] - 900s 2s/step - loss: 1.4401
Epoch 4/30
375/375 [==============================] - 910s 2s/step - loss: 1.3273
Epoch 5/30
375/375 [==============================] - 879s 2s/step - loss: 1.2679
Epoch 6/30
375/375 [==============================] - 884s 2s/step - loss: 1.2291
Epoch 7/30
375/375 [==============================] - 886s 2s/step - loss: 1.2001
Epoch 8/30
375/375 [==============================] - 895s 2s/step - loss: 1.1768
Epoch 9/30
375/375 [==============================] - 902s 2s/step - loss: 1.1571
Epoch 10/30
375/375 [==============================] - 984s 3s/step - loss: 1.1397
Epoch 11/30
375/375 [==============================] - 916s 2s/step - loss: 1.1232
Epoch 12/30
375/375 [==============================] - 933s 2s/step - loss: 1.1093
Epoch 13/30
3

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (120, None, 64)           5376      
                                                                 
 gru_1 (GRU)                 (120, None, 1026)         3361176   
                                                                 
 dense_1 (Dense)             (120, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
batch_size=1

In [98]:
model = create_model(rnn_size,emb_size,batch_size,vocab_size)


In [99]:
model.load_weights('MY_Shakesphere_model.h5')

model.build(tf.TensorShape([1,None]))

In [100]:
def generate_text(model,gen_size,start_seed,temp):
    generated_text=[]
    encod = [char_to_ind[c] for c in start_seed]
    encod=tf.expand_dims(encod,0)
    model.reset_states()
    for i in range(gen_size):
        predictions=model(encod)
        
        predictions=tf.squeeze(predictions,0)
        
        predictions = predictions/temp
        
        predictions_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        
        encod = tf.expand_dims([predictions_id], 0)
        
        generated_text.append(ind_to_char[predictions_id])
        
    return (start_seed + ''.join(generated_text))
        
    

In [101]:
print(generate_text(model,1000,'flower',0.1))

flower than the world.
    I will not have the world to see you.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [107]:
print(generate_text(model,2000,'Juliet',0.1))

Juliet.
  PROTEUS. The storm begins to seek his daughter.
    The time will be delivered to the world,
    And then to hear me speak a word with you.
    I will not have the world to see you.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [106]:
# loss_data = pd.DataFrame(model.history.history)

In [108]:
print(generate_text(model,2000,'India',0.1))

India strange things strange;
    And then I will not have the world to do
    And see the grace that hath most strong a part.
    I will not hear thee still for this offence.
    The wind was sent to th' way to come to me
    To th' strong that makes the world to see you.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      